## What is RNN

## Sequence Generation Scheme 

![alt text](https://miro.medium.com/max/3444/1*XvUt5wDQA8D3C0wAuxAvbA.png)

In [1]:
import tensorflow as tf
import string
import requests
import pandas as pd

In [2]:
response = requests.get('https://raw.githubusercontent.com/laxmimerit/poetry-data/master/adele.txt')

In [3]:
response

<Response [200]>

In [18]:
response.text

'Looking for some education\nMade my way into the night\nAll that bullshit conversation\nBaby, can\'t you read the signs? I won\'t bore you with the details, baby\nI don\'t even wanna waste your time\nLet\'s just say that maybe\nYou could help me ease my mind\nI ain\'t Mr. Right But if you\'re looking for fast love\nIf that\'s love in your eyes\nIt\'s more than enough\nHad some bad love\nSo fast love is all that I\'ve got on my mind Ooh, ooh\nOoh, ooh Looking for some affirmation\nMade my way into the sun\nMy friends got their ladies\nAnd they\'re all having babies\nI just wanna have some fun I won\'t bore you with the details, baby\nI don\'t even wanna waste your time\nLet\'s just say that maybe\nYou could help me ease my mind\nI ain\'t Mr. Right But if you\'re looking for fast love\nIf that\'s love in your eyes\nIt\'s more than enough\nI\'ve had some bad love\nSo fast love is all that I\'ve got on my mind Ooh, ooh\nBaby, baby\nOoh, ooh\nBaby, baby\nOoh, ooh\nBaby, baby\nOoh In the ab

In [19]:
data = response.text.splitlines()

In [20]:
len(data)

2400

In [24]:
len(" ".join(data))

91330

## Build LSTM Model and Prepare X and y

In [25]:
# tokenization

In [26]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [27]:
token = Tokenizer()
token.fit_on_texts(data)

In [29]:
#token.word_counts

In [30]:
#help(token)

In [31]:
token.word_index

{'i': 1,
 'you': 2,
 'the': 3,
 'me': 4,
 'to': 5,
 'it': 6,
 'and': 7,
 'my': 8,
 'your': 9,
 'that': 10,
 'love': 11,
 'of': 12,
 'in': 13,
 'all': 14,
 'we': 15,
 'a': 16,
 "i'm": 17,
 'when': 18,
 'but': 19,
 'be': 20,
 'for': 21,
 'so': 22,
 "don't": 23,
 'never': 24,
 "you're": 25,
 'let': 26,
 'if': 27,
 'is': 28,
 'know': 29,
 'up': 30,
 'go': 31,
 'down': 32,
 'will': 33,
 'on': 34,
 'just': 35,
 'do': 36,
 "ain't": 37,
 'heart': 38,
 'like': 39,
 'with': 40,
 'baby': 41,
 "it's": 42,
 'want': 43,
 'this': 44,
 "i've": 45,
 'say': 46,
 'no': 47,
 'as': 48,
 'had': 49,
 'make': 50,
 'can': 51,
 'out': 52,
 'ooh': 53,
 'was': 54,
 'one': 55,
 "can't": 56,
 'time': 57,
 'what': 58,
 'are': 59,
 'feel': 60,
 'see': 61,
 "i'll": 62,
 'fall': 63,
 'there': 64,
 'were': 65,
 'gonna': 66,
 'more': 67,
 "won't": 68,
 'oh': 69,
 'back': 70,
 'face': 71,
 'have': 72,
 'yeah': 73,
 'at': 74,
 "'cause": 75,
 'only': 76,
 'been': 77,
 'stand': 78,
 'ready': 79,
 'way': 80,
 'take': 81,
 'ha

In [32]:
encoded_text = token.texts_to_sequences(data)

In [39]:
encoded_text[0]

[254, 21, 219, 725]

In [35]:
x = ['i love you']

In [36]:
token.texts_to_sequences(x)

[[1, 11, 2]]

In [38]:
vocab_size = len(token.word_counts) + 1
vocab_size

1396

##  Prepare Training Data

In [41]:
datalist = []
for d in encoded_text:
  if len(d)>1:
    for i in range(2, len(d)):
      datalist.append(d[:i])

print(datalist[0:20])

[[254, 21], [254, 21, 219], [117, 8], [117, 8, 80], [117, 8, 80, 153], [117, 8, 80, 153, 3], [14, 10], [14, 10, 726], [41, 56], [41, 56, 2], [41, 56, 2, 603], [41, 56, 2, 603, 3], [41, 56, 2, 603, 3, 728], [41, 56, 2, 603, 3, 728, 1], [41, 56, 2, 603, 3, 728, 1, 68], [41, 56, 2, 603, 3, 728, 1, 68, 517], [41, 56, 2, 603, 3, 728, 1, 68, 517, 2], [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40], [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3], [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518]]


## Padding

In [42]:
max_length = 20
sequences = pad_sequences(datalist, maxlen=max_length, padding='pre')
sequences

array([[  0,   0,   0, ...,   0, 254,  21],
       [  0,   0,   0, ..., 254,  21, 219],
       [  0,   0,   0, ...,   0, 117,   8],
       ...,
       [  0,   0,   0, ...,  17, 198,  17],
       [  0,   0,   0, ..., 198,  17, 198],
       [  0,   0,   0, ...,  17, 198,   6]], dtype=int32)

In [43]:
X = sequences[:, :-1]

In [44]:
y = sequences[:, -1]

In [45]:
y

array([ 21, 219,   8, ...,  17, 198,   6], dtype=int32)

In [46]:
y = to_categorical(y, num_classes=vocab_size)

In [47]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [49]:
X.shape

(14231, 19)

In [50]:
seq_length = X.shape[1]

In [51]:
seq_length

19

## LSTM Model Training

In [52]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [54]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 50)            69800     
_________________________________________________________________
lstm (LSTM)                  (None, 19, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 1396)              140996    
Total params: 361,696
Trainable params: 361,696
Non-trainable params: 0
_________________________________________________________________


In [55]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [56]:
model.fit(X, y, batch_size=32, epochs=100)

Epoch 1/100
445/445 [==============================] - 13s 8ms/step - loss: 6.0361 - accuracy: 0.0378
Epoch 2/100
445/445 [==============================] - 4s 8ms/step - loss: 5.3161 - accuracy: 0.0488
Epoch 3/100
445/445 [==============================] - 4s 8ms/step - loss: 5.1409 - accuracy: 0.0555
Epoch 4/100
445/445 [==============================] - 4s 8ms/step - loss: 4.9328 - accuracy: 0.0637
Epoch 5/100
445/445 [==============================] - 4s 8ms/step - loss: 4.8220 - accuracy: 0.0771
Epoch 6/100
445/445 [==============================] - 4s 8ms/step - loss: 4.6413 - accuracy: 0.0917
Epoch 7/100
445/445 [==============================] - 4s 8ms/step - loss: 4.5330 - accuracy: 0.0985
Epoch 8/100
445/445 [==============================] - 4s 8ms/step - loss: 4.4160 - accuracy: 0.1091
Epoch 9/100
445/445 [==============================] - 4s 8ms/step - loss: 4.2846 - accuracy: 0.1191
Epoch 10/100
445/445 [==============================] - 4s 8ms/step - loss: 4.1885 - accur

## Poetry Generation

In [65]:
poetry_length = 10

def generate_poetry(seed_text, n_lines):

  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = token.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre')


      y_pred = np.argmax(model.predict(encoded), axis=-1)

      predicted_word = ""
      for word, index in token.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + ' ' + predicted_word
      text.append(predicted_word)

    seed_text = text[-1]
    text = ' '.join(text)
    print(text)




In [66]:
seed_text = 'when did you '
generate_poetry(seed_text, 5)

find the note that i will be hard until the
sweetest devotion i've known i've been looking for you i
can't give you what you think you gave you in
the single things and burst me into my heart burst
a part of little coming yeah yeah yeah yeah yeah
